In [2]:
#pip install paho-mqtt pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import paho.mqtt.client as mqtt
import pandas as pd
from datetime import datetime
import threading

# Broker settings
broker_address = "broker.hivemq.com"
port = 1883
topic_temp = "home/temperature/GroupTest"
topic_humidity = "home/humidity/GroupTest"

# Data buffer
data_buffer = {
    "timestamp": None,
    "temperature": None,
    "humidity": None
}

# Initialize DataFrame to store the complete data
df = pd.DataFrame(columns=["timestamp", "temperature", "humidity"])

# Callback when connecting to the MQTT broker
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe([(topic_temp, 0), (topic_humidity, 0)])

# Callback when receiving a message from the MQTT broker
def on_message(client, userdata, msg):
    message = msg.payload.decode()
    value = float(message.split(': ')[1])  # Split and convert

    if msg.topic == topic_temp:
        data_buffer['temperature'] = value
    elif msg.topic == topic_humidity:
        data_buffer['humidity'] = value

    # Check if both temperature and humidity are received
    if data_buffer['temperature'] is not None and data_buffer['humidity'] is not None:
        # Record timestamp when both values are available
        data_buffer['timestamp'] = datetime.now()

        # Append the data to the DataFrame
        df.loc[len(df)] = data_buffer

        # Reset buffer
        data_buffer['temperature'] = None
        data_buffer['humidity'] = None

def stop_collecting_data():
    # Save the DataFrame to CSV
    df.to_csv("mqtt_data.csv", index=False)
    print("Data collection completed and saved to CSV.")
    client.disconnect()

# Setup MQTT client and hooks
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

# Connect to the broker and start the loop
client.connect(broker_address, port=port)

# Set a timer for 600 seconds (10 minutes) to stop collecting data
timer = threading.Timer(600, stop_collecting_data)
timer.start()

client.loop_forever()


C:\Users\Eli\AppData\Local\Temp\ipykernel_1884\2926788564.py:56: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Connected with result code 0
Data collection completed and saved to CSV.


<MQTTErrorCode.MQTT_ERR_NO_CONN: 4>